In [ ]:
import sys
import time
import copy 
import os
import openpyxl
from openpyxl import Workbook
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image   #  pip install pillow
import glob
import torch
from torch.utils import data
import torchvision
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
print('PyTorch version:', torch.__version__) 
print('Python version:',sys.version)

In [ ]:
torch.cuda.is_available()

In [ ]:
imgs_path = glob.glob(r'Data/*/*.tif')
imgs_path[-3:]

In [ ]:
imgs_path[-3:]

In [ ]:
img_p = imgs_path[7]
img_p

In [ ]:
img_p.split('\\')[1]

In [ ]:
label_names = [img_p.split('\\')[1] for img_p in imgs_path]

In [ ]:
unique_label = np.unique(label_names)

In [ ]:
unique_label

In [ ]:
label_to_index = dict((v, k) for k, v in enumerate(unique_label))

In [ ]:
label_to_index

In [ ]:
index_to_label = dict((v, k) for k, v in label_to_index.items())

In [ ]:
index_to_label

In [ ]:
all_labels = [label_to_index.get(la) for la in label_names]

In [ ]:
all_labels[:5]

In [ ]:
all_labels[-5: ]

In [ ]:
len(imgs_path)

In [ ]:
np.random.seed(2023)
random_index = np.random.permutation(len(imgs_path))

In [ ]:
imgs_path = np.array(imgs_path)[random_index]
all_labels = np.array(all_labels)[random_index]

In [ ]:
imgs_path[:5]

In [ ]:
imgs_path[1].shape

# The data is divided into 60%, 30%, and 10% of the training set, test set, and validation set, respectively.

In [ ]:
train_len = int(len(imgs_path)*0.6)
eval_len = int(len(imgs_path)*0.2)

train_path = imgs_path[ :train_len]
train_labels = all_labels[ :train_len]

eval_path = imgs_path[train_len: train_len+eval_len]
eval_labels = all_labels[train_len:train_len+eval_len]

test_path = imgs_path[train_len+eval_len:]
test_labels = all_labels[train_len+eval_len:]

# Dataset creation

In [ ]:
train_transform = transforms.Compose([                   
                    transforms.Resize((1024, 1024)),
                    #transforms.CenterCrop((680,680)),
                    #transforms.RandomRotation(degrees=(40)),  
                    transforms.ToTensor(),   
                    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])# mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5]
])

eval_transform = transforms.Compose([
                    transforms.Resize((1024, 1024)),
                    #transforms.CenterCrop((680,680)),
                    #transforms.RandomRotation(degrees=(40)),   
                    transforms.ToTensor(), 
                    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])# mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5]
])

test_transform = transforms.Compose([
                    transforms.Resize((1024, 1024)),
                    #transforms.CenterCrop((680,680)),
                    #transforms.RandomRotation(degrees=(40)),   
                    transforms.ToTensor(), 
                    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])# mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5]
])

In [ ]:
#Create a subclass of the Dataset
class Mydataset(data.Dataset):       
    def __init__(self, img_paths, labels,transform):  #Initialize the path to the image
        self.imgs = img_paths
        self.labels = labels
        self.transforms = transform

    def __getitem__(self, index):   #Implement data slicing
        img = self.imgs[index]
        label = self.labels[index]

        #pil_img = Image.open(img)    
        pil_img = glob.glob(img)
        pil_img=Image.fromarray(pil_img)
        
        data = self.transforms(pil_img)

        return data, label

    def __len__(self):            #Returns the total length of the data
        return len(self.imgs)

In [ ]:
train_ds = Mydataset(train_path, train_labels, train_transform)
eval_ds = Mydataset(eval_path, eval_labels, eval_transform)
test_ds = Mydataset(test_path, test_labels, test_transform)

In [ ]:
BATCH_SIZE =10

In [ ]:
train_dl = data.DataLoader(
                           train_ds,
                           batch_size=BATCH_SIZE,
                           shuffle=True
                           )

eval_dl = data.DataLoader(
                          eval_ds,
                          batch_size=BATCH_SIZE
                          )

test_dl = data.DataLoader(
                          test_ds,
                          batch_size=BATCH_SIZE
                          )

In [ ]:
imgs_batch, labels_batch = next(iter(train_dl))

In [ ]:
imgs_batch.shape

In [ ]:
labels_batch.shape

In [ ]:
plt.figure(figsize=(18, 12))

for i, (img, label) in enumerate(zip(imgs_batch[-6:], labels_batch[-6:])):
    #img = img.permute(1, 2, 0).numpy()
    img = (img.permute(1, 2, 0).numpy()+1)/2    
    plt.subplot(2, 3, i+1)
    plt.title(index_to_label.get(label.item()))
    plt.imshow(img)

In [ ]:
im=imgs_batch[0].permute(1,2,0)
im.shape

In [ ]:
#im=im.numpy()
img = (im+1)/2   

In [ ]:
type(im)

In [ ]:
plt.imshow(im)

# Construct FSVNet model - use CNN to extract features

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 16, 3)
        self.bn1=nn.BatchNorm2d(16)
        
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.bn2=nn.BatchNorm2d(32)
        
        self.conv3 = nn.Conv2d(32, 64, 3)
        self.bn3=nn.BatchNorm2d(64)
        
        self.conv4 = nn.Conv2d(64, 128, 3)
        self.bn4=nn.BatchNorm2d(128)
        
        self.conv5 = nn.Conv2d(128,256,3)
        self.bn5=nn.BatchNorm2d(256)
        
        self.conv6 = nn.Conv2d(256,512,3)
        self.bn6=nn.BatchNorm2d(512)
        
        self.conv7 = nn.Conv2d(512,1024,3)
        self.bn7=nn.BatchNorm2d(1024)

        self.conv8 = nn.Conv2d(1024,2048,3)
        self.bn8=nn.BatchNorm2d(2048)
    
        
        self.pool = nn.MaxPool2d(2, 2)
        self.drop = nn.Dropout(0.3)
        
        self.fc1 = nn.Linear(2048*2*2, 4096)   
        self.bn_f1=nn.BatchNorm1d(4096)  
        self.fc2 = nn.Linear(4096, 2048)
        self.bn_f2=nn.BatchNorm1d(2048) 
        self.fc3 = nn.Linear(2048, 1024) 
        self.bn_f3=nn.BatchNorm1d(1024) 
        self.fc4 = nn.Linear(1024,512)  
        self.bn_f4=nn.BatchNorm1d(512) 
        self.fc5 = nn.Linear(512, 256) 
        self.bn_f5=nn.BatchNorm1d(256) 
        self.fc6 = nn.Linear(256, 128)  
        self.bn_f6=nn.BatchNorm1d(128) 
        self.fc7= nn.Linear(128, 2)  
        
    def forward(self, x):
        x=self.pool(F.relu(self.conv1(x)))
        x=self.bn1(x)
        x=self.pool(F.relu(self.conv2(x)))
        x=self.bn2(x)
        x=self.pool(F.relu(self.conv3(x)))
        x=self.bn3(x)
        x=self.pool(F.relu(self.conv4(x)))
        x=self.bn4(x)  
        x=self.pool(F.relu(self.conv5(x)))
        x=self.bn5(x)
        x=self.pool(F.relu(self.conv6(x)))
        x=self.bn6(x)  
        x=self.pool(F.relu(self.conv7(x))) 
        x=self.bn7(x) 
        x=self.pool(F.relu(self.conv8(x))) 
        x=self.bn8(x)        
        #x=self.pool(F.relu(self.conv9(x))) 
        #x=self.bn9(x) 
        #x=self.pool(F.relu(self.conv10(x))) 
        #x=self.bn10(x)         
        x=self.drop(x)        
        #print(x.size())
        x=x.view(-1, 2048*2*2)        
        #x=x.view(-1, x.size(1)*x.size(2)*x.size(3))       
        #print(x.size())
        x=F.relu(self.fc1(x))
        #print(x.size())
        
        x=self.bn_f1(x)        
        x=self.drop(x) 
        
        x=F.relu(self.fc2(x))
        x=self.bn_f2(x)
        x=self.drop(x) 
        
        x=F.relu(self.fc3(x))
        x=self.bn_f3(x)
        x=self.drop(x)
        
        x=F.relu(self.fc4(x))
        x=self.bn_f4(x)
        x=self.drop(x)

        x=F.relu(self.fc5(x))
        x=self.bn_f5(x)
        x=self.drop(x)

        x=F.relu(self.fc6(x))
        x=self.bn_f6(x)
        x=self.drop(x)

        x=F.relu(self.fc7(x))
       
        return x

# CUDA initialization

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available else "cpu")
device

# Model initialization

In [ ]:
model=Net()
model.to(device)

# The amount of model computation and parameters

In [ ]:
#1：
from torchsummary import summary

#print(summary(model, input_size=(3, 1024, 1024)))

In [ ]:
#2：
#total = sum([param.nelement() for param in model.parameters()]) 
#print("Number of parameter: %.2fM" % (total/1e6))

In [ ]:
#3
from thop import profile

#input = torch.randn(1, 3,1024,1024)
#flops, params = profile(model, inputs=(input,))
#print('flops: ', flops, 'params: ', params)
#print(' flops: %.2f M, params: %.2f M' % (flops //1e6, params //1e6))

In [ ]:
#model(imgs_batch)

# Train the model

In [ ]:
#Training function
def train_model(model, train_dataloader, criterion, optimizer, device):

    correct=0
    total=0
    running_loss = 0.0

    start_time = time.time()    

    model.train()
    for inputs, labels in train_dataloader:
        labels=torch.as_tensor(labels,dtype=torch.long)
        inputs, labels = inputs.to(device), labels.to(device)        

        predict = model(inputs)
        loss = criterion(predict,labels) 

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            predict=torch.argmax(predict,dim=1)
            correct+=(predict==labels).sum().item()
            total+=labels.size(0)
            running_loss+=loss.item()

    loss = running_loss / len(train_dataloader.dataset)
    acc = correct / total

    train_time = time.time() - start_time 

    return loss,acc

# Validate the model

In [ ]:
#Validate model functions
def eval_model(model, eval_dataloader,criterion, device):

    eval_correct = 0
    eval_total = 0
    eval_running_loss = 0.0

    predictions = []
    targets = []

    start_time = time.time()    

    model.eval()
    with torch.no_grad():
        for inputs, labels in eval_dataloader:
            labels=torch.as_tensor(labels,dtype=torch.long)
            inputs,labels = inputs.to(device),labels.to(device)

            predict = model(inputs)
            loss = criterion(predict, labels) 

            predict=torch.argmax(predict,dim=1)

            eval_correct+=(predict==labels).sum().item()
            eval_total+=labels.size(0)
            eval_running_loss+=loss.item()

            predictions.extend(predict.cpu().numpy())
            targets.extend(labels.cpu().numpy())

    loss = eval_running_loss / len(eval_dataloader.dataset)
    acc = eval_correct / eval_total

    eval_time = time.time() - start_time 

    return loss, acc, predictions, targets

# Test the model

In [ ]:
# Test the model function
def test_model(model, test_loader,device):
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    print(f'Test Accuracy: {100 * correct / total:.2f}%')

    save_dir='DataReport'
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)

    book = openpyxl.Workbook()
    sheet = book.active
    sheet.title = "DataReport"

    sheet.append(['ground truth', 'preds'])

    for i in range(0, len(all_preds)):
        sheet.append([all_labels[i],all_preds[i]])

    book.save(save_dir+"/FsvNet_labels_preds.xlsx")

    return  all_labels,all_preds

# Start training and validation

In [ ]:
#Repeat the training rounds
epochs = 20

# Define the loss function and optimizer  
criterion = torch.nn.CrossEntropyLoss()   #criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

#Model data logging
best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0

#Training data logging
train_loss = []
train_acc = []
test_loss = []
test_acc = []

#Evaluate data logging
test_preds = []
test_labels = []  

# Training and evaluation
start_time = time.time()    

for epoch in range(epochs):
    #model train
    epoch_train_loss,epoch_train_acc = train_model(model, train_dl, criterion, optimizer, device)           

    # test and evaluate
    epoch_eval_loss,epoch_eval_acc,eval_predictions, eval_targets = eval_model(model, eval_dl,criterion, device)

    #loss and acc. output
    print(f"Epoch {epoch+1}/{epochs}: Train Loss: {epoch_train_loss:.4f}   Train Acc: {epoch_train_acc:.4f}  Eval_Loss：{epoch_eval_loss:.4f}   Eval_Acc:{epoch_eval_acc:.4f}")

    if epoch_eval_acc>best_acc:
        best_acc=epoch_eval_acc
        best_model_wts=copy.deepcopy(model.state_dict())

    train_loss.append(epoch_train_loss)
    train_acc.append(epoch_train_acc)
    test_loss.append(epoch_eval_loss)
    test_acc.append(epoch_eval_acc)        


model.load_state_dict(best_model_wts) 

duration = (time.time() - start_time)/60

print(f"Model_best_acc: {best_acc:.4f} ,Lasted {duration:.0f}min")

# Model parameters are saved

In [ ]:
#Save the model weights to a file
PATH='FscvNet_Weights.pth'
torch.save(model.state_dict(),PATH)
print("Model weights saved.")

# Performance plotting

In [ ]:
#loss plotting
plt.plot(range(1, epochs+1), train_loss, label='train_loss')
plt.plot(range(1, epochs+1), test_loss, label='test_loss')
plt.legend()

# save
save_dir='DataReport'
if not os.path.isdir(save_dir):
    os.mkdir(save_dir)

plt.savefig(save_dir+'BaseCnn_loss.png')

plt.show()

In [ ]:
#acc. plotting
plt.plot(range(1, epochs+1), train_acc, label='train_acc')
plt.plot(range(1, epochs+1), test_acc, label='test_acc')
plt.legend()

# save
save_dir='DataReport'
if not os.path.isdir(save_dir):
    os.mkdir(save_dir)

plt.savefig(save_dir+'BaseCnn_Acc.png')

plt.show()

In [ ]:
#Loss, accuracy data storage
save_dir='DataReport'
if not os.path.isdir(save_dir):
    os.mkdir(save_dir)

book = openpyxl.Workbook()
sheet = book.active
sheet.title = "DataReport"

sheet.append(['epochs', 'train_loss','test_loss','train_acc','test_acc'])

for i in range(0, epochs):
    sheet.append([i+1, train_loss[i], test_loss[i],train_acc[i],test_acc[i]])

book.save(save_dir+"/BaseCnn_Loss_Accuracy.xlsx")


#range(0,epochs).to_excel(save_dir+"/ResNet_Loss_Accuracy.xlsx", encoding="utf_8_sig")
#train_loss.to_excel(save_dir+"/ResNet_Loss_Accuracy.xlsx", encoding="utf_8_sig")
#test_loss.to_excel(save_dir+"/ResNet_Loss_Accuracy.xlsx", encoding="utf_8_sig")
#test_loss.to_excel(save_dir+"/ResNet_Loss_Accuracy.xlsx", encoding="utf_8_sig")
#test_acc.to_excel(save_dir+"/ResNet_Loss_Accuracy.xlsx", encoding="utf_8_sig")

# Model parameter import

In [ ]:
model=Net()
model.load_state_dict(torch.load(PATH))
print("Model weights reloaded.")

# Model testing

In [ ]:
model.to(device)
labels,preds=test_model(model, test_dl, device)

# Calculate the confusion matrix

In [ ]:
cm = confusion_matrix(labels, preds)
cm

# Draw the confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
# cm = confusion_matrix(y_true=y_test, y_pred=y_pred, normalize='true')


def plot_confusion_matrix(cm,labels,preds): 

    # confusion matrix
    class_names = ['signal1','signal2']
    title="confusion_matrix"
    ylabel='Ground Truth'
    xlabel='Predicted Label'

    tick_marks = np.arange(len(class_names))

    #plt.figure(figsize=(18, 12))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.BuPu_r)  
    #plt.imshow(cm, interpolation='nearest',cmap='RdBu')
    #plt.colorbar(label='tota1 test sample number')    

    plt.title(title)
    plt.xticks(tick_marks,class_names,rotation=45)
    plt.yticks(tick_marks,class_names)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)

     # save
    save_dir='DataReport'
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)

    plt.savefig(save_dir+'BaseCnn_Confusion_Matrix.png')

    plt.show()

In [ ]:
plot_confusion_matrix(cm,labels,preds)

In [ ]:
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm,labels,preds): 

    # plot
    class_names = ['signal1','signal2']
    title="confusion_matrix"
    ylabel='Ground Truth'
    xlabel='Predicted Label'

    tick_marks = np.arange(len(class_names))

    #plt.figure(figsize=(18, 12))
    #plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)  
    plt.imshow(cm, interpolation='nearest',cmap='BuPu')
    #plt.colorbar(label='tota1 test sample number')    

    plt.title(title)
    plt.xticks(tick_marks,class_names,rotation=45)
    plt.yticks(tick_marks,class_names)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)

     # save
    save_dir='DataReport'
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)

    plt.savefig(save_dir+'BaseCnn_Confusion_Matrix.png')

    plt.show()

In [ ]:
plot_confusion_matrix(cm,labels,preds)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(labels,preds)
class_names = ['signal1','signal2']
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
#disp.plot() 
disp.plot(cmap=plt.cm.Blues_r)


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(labels,preds)
class_names = ['signal1','signal2']
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap=plt.cm.BuPu)
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(labels,preds)
class_names = ['signal1','signal2']
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap=plt.cm.PuBu)
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(labels,preds)
class_names = ['signal1','signal2']
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap=plt.cm.PuBu_r)
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

class_names = ['signal1','signal2']
title="confusion_matrix"
ylabel='Ground Truth'
xlabel='Predicted Label'

tick_marks = np.arange(len(class_names))

cm=confusion_matrix(labels, preds)

#sns.heatmap(cm, annot=True) 
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False) 
#sns.heatmap(cm, annot=True,cmap='PuOr') 
#sns.heatmap(cm, cmap="YlGnBu",annot=True)

plt.xlabel(xlabel,fontsize=20, color='k') 
plt.ylabel(ylabel,fontsize=20, color='k') 
plt.xticks(tick_marks,class_names,rotation=45) 
plt.yticks(tick_marks,class_names)  
plt.title(title,fontsize=20) 
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

class_names = ['signal1','signal2']
title="confusion_matrix"
ylabel='Ground Truth'
xlabel='Predicted Label'

tick_marks = np.arange(len(class_names))

cm=confusion_matrix(labels, preds)

#sns.heatmap(cm, annot=True) 
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False) 
#sns.heatmap(cm, annot=True,cmap='PuOr') 
#sns.heatmap(cm, cmap="YlGnBu",annot=True)

plt.xlabel(xlabel,fontsize=20, color='k') 
plt.ylabel(ylabel,fontsize=20, color='k') 
plt.xticks(tick_marks,class_names,rotation=45) 
plt.yticks(tick_marks,class_names)  
plt.title(title,fontsize=20) 
plt.show()

# Comprehensive evaluation of model performance

In [ ]:
# Calculate evaluation metrics  
from sklearn.metrics import f1_score, mean_absolute_error, mean_squared_error, roc_auc_score  
from sklearn.metrics import accuracy_score, precision_score, recall_score, ndcg_score 
 
# Define the function for calculating F1-score 
def calculate_f1_score(true_labels, predicted_labels): 
 return f1_score(true_labels, predicted_labels) 
 
# Define the function for calculating MAE 
def calculate_mae(true_ values, predicted_values): 
 return mean_absolute_error(true_values, predicted_values) 
 
# Define the function to calculate RMSE 
def calculate_rmse(true_values, predicted_values): 
 return np.sqrt(mean_ squared_error(true_values, predicted_values)) 
 
 
# Define the function for calculating Accuracy 
def calculate_accuracy(true_labels, predicted_labels): 
 return accuracy_score(true_labels, predicted_labels 
 
# Calculate Precision@k metrics 
def calculate_precision(true_labels, predicted_labels):  
 return precision_score(true_labels, predicted_labels) 
 
# Define the function to calculate Recall 
def calculate_recall(true_ labels, predicted_labels): 
 return recall_score(true_labels, predicted_labels) 
 
 
# Define the function for calculating ROC AUC  
def calculate_roc_auc(true_labels, predicted_scores): 
 return roc_auc_score( true_labels, predicted_scores) 
 
# Calculate NDCG@k metrics  
def compute_ndcg(predictions, targets, k):  
 ndcg_scores = ndcg_score(targets, predictions, k=k) 
 return ndcg_scores.mean() 
 
# Calculate Hit Rate@k Indicators  
def compute_hit_rate(predictions, targets, k): 
 num_hits = 0 
 for i in range(len(predictions)): 
 if targets[i] in predictions[i, :k]: 
 num_hits += 1 
 return num_hits /  len(predictions) 
 
# Calculate Precision@k metrics  
def compute_precision(predictions, targets, k): 
 precision_scores = [] 
 for i in range(len(predictions)): 
 precision = precision_score([targets[ i]], predictions[i, :k], average='micro') 
        precision_scores.append(precision) 
    return sum(precision_scores) / len(predictions)

In [ ]:
# evaluation
print("F1-score:", calculate_f1_score(labels, preds))
print("Accuracy:", calculate_accuracy(labels, preds))
print("Precision:", calculate_precision(labels, preds))
print("Recall:", calculate_recall(labels, preds))
print("ROC_AUC:", calculate_roc_auc(labels, preds))

#print(f"Precision@{k}: {precision:.4f}")

# ROC curves and AUC were calculated

In [ ]:
#ROC（Receiver Operating Characteristic） AUC（Area Under the ROC Curve）

from sklearn.metrics import roc_curve, auc, roc_auc_score

# ROC curves and AUC 
fpr, tpr, thresholds = roc_curve(labels, preds)
roc_auc = roc_auc_score(labels, preds) #roc_auc = auc(fpr, tpr)

# Visualize the ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")

# save
save_dir='DataReport'
if not os.path.isdir(save_dir):
    os.mkdir(save_dir)

plt.savefig(save_dir+'BaseCnn_AUC.png')

plt.show()

print("AUC:", roc_auc)

# Instance testing

In [ ]:
# Instantiate the model
model=Net()
model.load_state_dict(torch.load(PATH))
model.to(device)

In [ ]:
#Test data preprocessing functions
def load_and_preprocess_image(img_path):
    transform_test = transforms.Compose([
                    transforms.Resize((1024, 1024)),
                    transforms.ToTensor()                    
    ])

    #image = glob.glob(img_path)
    image = Image.open(img_path)   
    image_tensor = transform_test(image)
    
    # Add a batch dimension to a single image
    image_tensor=torch.unsqueeze(image_tensor, 0)  #image_tensor.unsqueeze(0)

    return image_tensor

In [ ]:
1.Test a single image, load the unprocessed image directly for testing
image_path = 'Data_Val/1.tif'  #Replace with your image path
image = Image.open(image_path)
plt.imshow(image)

In [ ]:
width, height = image.size
channels = 1 if image.mode == 'L' else 3  # There is 1 channel for grayscale images and 3 channels for color images
print(f"Image dimensions: {width} x {height} x {channels}")

In [ ]:
image = load_and_preprocess_image(image_path).to(device)

model.eval() 
output = model(image)
#The category with the highest probability of output
_, predicted_class = torch.max(output, dim=1)
predicted_class

In [ ]:
percentage = F.softmax(output, dim=1)[0] * 100
percentage

In [ ]:
# Get the prediction results and sort them from largest to smallest
_, indices = torch.sort(output, descending=True)
indices

In [ ]:
#2. Select a random image for the DataLoader in the preprocessed dataset for testing
image,label = next(iter(test_dl))

y_pred = model(image.to(device)  )
y_pred = torch.argmax(y_pred, dim=1)

label,y_pred.cpu().numpy()

In [ ]:
image.shape

In [ ]:
sample=image[0].squeeze() #Remove the batch dimension
sample.shape

In [ ]:
sample = (sample.permute(1, 2, 0).numpy()+1)/2  
plt.title(label)
plt.imshow(sample)
plt.show()

In [ ]:
#3. Test a random image from the preprocessed dataset DataSet
import random
r_index = random.choice(range(len(test_ds)))

x, y = test_ds[r_index]

x=x.unsqueeze(0)  #x=torch.unsqueeze(x, 0)
y = torch.as_tensor(y, dtype=torch.long)

x, y = x.to(device), y.to(device)       

y_pred = model(x)
y_pred = torch.argmax(y_pred, dim=1)

y.cpu().numpy(),y_pred.cpu().numpy()